# Run pathways in FaIR

The pathways are generated elsewhere, imported here and then run.

In [1]:
import json
from multiprocessing import Pool

from climateforcing.utils import mkdir_p
import fair
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open('../data_input/fair-1.6.2-ar6/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

In [3]:
emissions_in = {}
results_out = {}
WORKERS = 3  # set this based on your individual machine - allows parallelisation. nprocessors-1 is a sensible shout.

In [4]:
scenarios = ["ssp245", "ch4_30", "ch4_40", "ch4_50", "ch4_100", "co2afolu_0", "coal-phase-out", "all"]

In [5]:
for scenario in scenarios:
    emissions_in[scenario] = np.loadtxt('../data_output/fair_emissions_files/{}.csv'.format(scenario), delimiter=',')

In [6]:
mkdir_p('../data_output/fair_temperatures/')

## convenience function for running FaIR config with each emission species

In [7]:
def run_fair(args):
    thisC, thisF, thisT, _, thisOHU, _, thisAF = fair.forward.fair_scm(**args)
    return (thisC[:,0], thisT, np.sum(thisF[:,:43], axis=1), np.sum(thisF[:,35:41], axis=1), np.sum(thisF, axis=1))

def fair_process(emissions):
    updated_config = []
    for i, cfg in enumerate(config_list):
        updated_config.append({})
        for key, value in cfg.items():
            if isinstance(value, list):
                updated_config[i][key] = np.asarray(value)
            else:
                updated_config[i][key] = value
        updated_config[i]['emissions'] = emissions
        updated_config[i]['diagnostics'] = 'AR6'
        updated_config[i]["efficacy"] = np.ones(45)
        updated_config[i]["gir_carbon_cycle"] = True
        updated_config[i]["temperature_function"] = "Geoffroy"
        updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
        updated_config[i]["fixPre1850RCP"] = False
    #    updated_config[i]["scale"][43] = 0.6
        updated_config[i]["F_solar"][270:] = 0
    
    if __name__ == '__main__':
        with Pool(WORKERS) as pool:
            result = list(tqdm(pool.imap(run_fair, updated_config), total=len(updated_config), position=0, leave=True))
    
    result_t = np.array(result).transpose(1,2,0)
    c, t, f_ant, f_aer, f_tot = result_t
    temp_rebase = t - t[100:151,:].mean(axis=0)
    
    return c, temp_rebase, f_ant, f_aer, f_tot

## Do the runs

In [8]:
for scenario in tqdm(scenarios, position=0, leave=True):
    results_out[scenario] = {}
    (
        results_out[scenario]['co2_conc'],
        results_out[scenario]['temp'],
        results_out[scenario]['F_ant'],
        results_out[scenario]['F_aer'],
        results_out[scenario]['F_tot']
    ) = fair_process(emissions_in[scenario])

100%|████████████████████████████████████████████| 8/8 [26:50<00:00, 201.35s/it]


## Save temperature outputs to analyse elsewhere

In [9]:
for scenario in scenarios:
    df_out = pd.DataFrame(results_out[scenario]['temp'][245:351,:])
    df_out['year'] = np.arange(1995.5, 2101)
    df_out.set_index('year', inplace=True)
    df_out.to_csv('../data_output/fair_temperatures/{}.csv'.format(scenario), float_format="%6.4f")